In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pprint

Note: you may need to restart the kernel to use updated packages.


In [3]:
import psycopg2
import pprint

In [4]:
dbname = 'hvvolzee'
user = 'hvvolzee'
password = 'dAKXw5LOi5bSb_IZtRP6yXMG0jddM_qD'  # Don't commit or share this for security purposes!
host = 'rajje.db.elephantsql.com'  # Port should be included or default
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)
#print(pg_conn)
pg_curs = pg_conn.cursor()

In [5]:
import sqlite3
import pandas as pd

df = pd.read_csv('/Users/julie/Desktop/repos/DS-Unit-3-Sprint-2-SQL-and-Databases/module2-sql-for-analysis/titanic.csv')
print(df.head())
print(df.shape)

df['Name']= df['Name'].str.replace("'","")
print(df.head())


   Survived  Pclass                                               Name  \
0         0       3                             Mr. Owen Harris Braund   
1         1       1  Mrs. John Bradley (Florence Briggs Thayer) Cum...   
2         1       3                              Miss. Laina Heikkinen   
3         1       1        Mrs. Jacques Heath (Lily May Peel) Futrelle   
4         0       3                            Mr. William Henry Allen   

      Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0    male  22.0                        1                        0   7.2500  
1  female  38.0                        1                        0  71.2833  
2  female  26.0                        0                        0   7.9250  
3  female  35.0                        1                        0  53.1000  
4    male  35.0                        0                        0   8.0500  
(887, 8)
   Survived  Pclass                                               Name  \
0         

In [6]:
df_conn=sqlite3.connect('titanic.sqlite3')
df_curs = df_conn.cursor()
df.to_sql('titanic',df_conn,if_exists='replace')
titanic=df_curs.execute('SELECT * FROM titanic').fetchall()
print(df_curs.execute('PRAGMA table_info(titanic);').fetchall())

[(0, 'index', 'INTEGER', 0, None, 0), (1, 'Survived', 'INTEGER', 0, None, 0), (2, 'Pclass', 'INTEGER', 0, None, 0), (3, 'Name', 'TEXT', 0, None, 0), (4, 'Sex', 'TEXT', 0, None, 0), (5, 'Age', 'REAL', 0, None, 0), (6, 'Siblings/Spouses Aboard', 'INTEGER', 0, None, 0), (7, 'Parents/Children Aboard', 'INTEGER', 0, None, 0), (8, 'Fare', 'REAL', 0, None, 0)]


/Users/julie/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [7]:
create_titanic_table ="""
CREATE TABLE IF NOT EXISTS Titanic(
    index INT,
    Survived INT,
    Pclass INT,
    Name TEXT,
    Sex Text,
    Age REAL,
    Siblings_Spouses_Aboard INT,
    Parents_Children_Aboard INT,
    Fare REAL

);"""

pg_curs.execute(create_titanic_table)
pg_conn.commit()
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()


[('public',
  'charactercreator_character',
  'hvvolzee',
  None,
  True,
  False,
  False,
  False),
 ('public', 'titanic', 'hvvolzee', None, False, False, False, False)]

In [8]:
for x in titanic:
  insert_x = """
    INSERT INTO Titanic
    (Survived, Pclass, Name, Sex,Age, Siblings_Spouses_Aboard,  Parents_Children_Aboard, Fare)
    VALUES """ + str(x[1:]) + ";"
  pg_curs.execute(insert_x)

# What was the average age of each passenger class?

In [9]:
pg_curs.execute('SELECT avg(age), pclass FROM Titanic WHERE survived = 1 GROUP BY pclass')
rows = pg_curs.fetchall()

In [10]:
pprint.pprint(rows)

[(35.8601518988609, 1), (21.4764571427447, 3), (26.3006274508495, 2)]


In [11]:
for row in rows:  
    print("avg age:", row[0], "class:", row[1])

avg age: 35.8601518988609 class: 1
avg age: 21.4764571427447 class: 3
avg age: 26.3006274508495 class: 2


# How many passengers survived, and how many died?

In [12]:
pg_curs.execute('SELECT count(*),Survived FROM Titanic GROUP BY Survived')
num = pg_curs.fetchall()
pprint.pprint(num)

[(1595, 0), (1000, 1)]


In [13]:
print("Total number of passengers survived :", num[0][0])
print("Total number of passengers died :", num[1][0])

Total number of passengers survived : 1595
Total number of passengers died : 1000


# How many passengers were in each class?

In [14]:
pg_curs.execute('SELECT count(*),Pclass FROM Titanic GROUP BY Pclass')
num1 = pg_curs.fetchall()
pprint.pprint(num1)

[(633, 1), (1423, 3), (539, 2)]


In [15]:
for n in num1:  
    print("Numbers of passengers:", n[0], "class:", n[1])

Numbers of passengers: 633 class: 1
Numbers of passengers: 1423 class: 3
Numbers of passengers: 539 class: 2


# How many passengers survived/died within each class?

In [16]:
pg_curs.execute('SELECT count(*),Survived,Pclass FROM Titanic GROUP BY Survived,Pclass')
num2 = pg_curs.fetchall()
pprint.pprint(num2)

[(1073, 0, 3), (238, 0, 1), (284, 0, 2), (350, 1, 3), (255, 1, 2), (395, 1, 1)]


In [17]:
print("Total number of passengers died in class 1:", num2[1][0])
print("Total number of passengers died in class 2:", num2[2][0])
print("Total number of passengers died in class 3:", num2[0][0])
print("Total number of passengers survived in class 1:", num2[5][0])
print("Total number of passengers survived in class 2:", num2[4][0])
print("Total number of passengers survived in class 3:", num2[3][0])

Total number of passengers died in class 1: 238
Total number of passengers died in class 2: 284
Total number of passengers died in class 3: 1073
Total number of passengers survived in class 1: 395
Total number of passengers survived in class 2: 255
Total number of passengers survived in class 3: 350


# What was the average age of survivors vs nonsurvivors?

In [18]:
pg_curs.execute('SELECT AVG(age),Survived FROM Titanic GROUP BY Survived')
num3 = pg_curs.fetchall()
pprint.pprint(num3)

[(30.2188087774295, 0), (28.3881799999773, 1)]


In [19]:
print("Average age of passengers survived :", num3[0][0])
print("Average age of passengers died :", num3[1][0])

Average age of passengers survived : 30.2188087774295
Average age of passengers died : 28.3881799999773


# What was the average fare by passenger class? By survival?

In [20]:
pg_curs.execute('SELECT avg(Fare),Survived,Pclass FROM Titanic GROUP BY Survived,Pclass')
num4 = pg_curs.fetchall()
pprint.pprint(num4)

[(13.6470823119093, 0, 3),
 (64.9943947912264, 0, 1),
 (19.4770683302006, 0, 2),
 (13.6097377967834, 1, 3),
 (22.0546086666631, 1, 2),
 (96.8168477263632, 1, 1)]


In [21]:
print("Average fare of passengers survived in class 1 :", num4[5][0])
print("Average fare of passengers survived in class 2 :", num4[4][0])
print("Average fare of passengers survived in class 3 :", num4[3][0])
print("Average fare of passengers died in class 1 :", num4[1][0])
print("Average fare of passengers died in class 2 :", num4[2][0])
print("Average fare of passengers died in class 3 :", num4[0][0])

Average fare of passengers survived in class 1 : 96.8168477263632
Average fare of passengers survived in class 2 : 22.0546086666631
Average fare of passengers survived in class 3 : 13.6097377967834
Average fare of passengers died in class 1 : 64.9943947912264
Average fare of passengers died in class 2 : 19.4770683302006
Average fare of passengers died in class 3 : 13.6470823119093


# How many siblings/spouses aboard on average, by passenger class? By survival?

In [22]:
pg_curs.execute('SELECT avg(Siblings_Spouses_Aboard),Survived,Pclass FROM Titanic GROUP BY Survived,Pclass')
num5 = pg_curs.fetchall()
pprint.pprint(num5)

[(Decimal('0.67660764212488350419'), 0, 3),
 (Decimal('0.28991596638655462185'), 0, 1),
 (Decimal('0.32042253521126760563'), 0, 2),
 (Decimal('0.44000000000000000000'), 1, 3),
 (Decimal('0.49411764705882352941'), 1, 2),
 (Decimal('0.49873417721518987342'), 1, 1)]


In [23]:
print("Average Siblings_Spouses_Aboard survived in class 1 :", num5[5][0])
print("Average Siblings_Spouses_Aboard survived in class 2 :", num5[4][0])
print("Average Siblings_Spouses_Aboard survived in class 3 :", num5[3][0])
print("Average Siblings_Spouses_Aboard died in class 1 :", num5[1][0])
print("Average Siblings_Spouses_Aboard died in class 2 :", num5[2][0])
print("Average Siblings_Spouses_Aboard died in class 3 :", num5[0][0])

Average Siblings_Spouses_Aboard survived in class 1 : 0.49873417721518987342
Average Siblings_Spouses_Aboard survived in class 2 : 0.49411764705882352941
Average Siblings_Spouses_Aboard survived in class 3 : 0.44000000000000000000
Average Siblings_Spouses_Aboard died in class 1 : 0.28991596638655462185
Average Siblings_Spouses_Aboard died in class 2 : 0.32042253521126760563
Average Siblings_Spouses_Aboard died in class 3 : 0.67660764212488350419


# How many parents/children aboard on average, by passenger class? By survival?

In [24]:
pg_curs.execute('SELECT avg(Parents_Children_Aboard),Survived,Pclass FROM Titanic GROUP BY Survived,Pclass')
num6 = pg_curs.fetchall()
pprint.pprint(num6)

[(Decimal('0.38676607642124883504'), 0, 3),
 (Decimal('0.30252100840336134454'), 0, 1),
 (Decimal('0.14436619718309859155'), 0, 2),
 (Decimal('0.41428571428571428571'), 1, 3),
 (Decimal('0.64313725490196078431'), 1, 2),
 (Decimal('0.38987341772151898734'), 1, 1)]


In [25]:
print("Average Parents_Children_Aboard survived in class 1 :", num5[5][0])
print("Average Parents_Children_Aboard survived in class 2 :", num5[4][0])
print("Average Parents_Children_Aboard survived in class 3 :", num5[3][0])
print("Average Parents_Children_Aboard died in class 1 :", num5[1][0])
print("Average Parents_Children_Aboard died in class 2 :", num5[2][0])
print("Average Parents_Children_Aboard died in class 3 :", num5[0][0])

Average Parents_Children_Aboard survived in class 1 : 0.49873417721518987342
Average Parents_Children_Aboard survived in class 2 : 0.49411764705882352941
Average Parents_Children_Aboard survived in class 3 : 0.44000000000000000000
Average Parents_Children_Aboard died in class 1 : 0.28991596638655462185
Average Parents_Children_Aboard died in class 2 : 0.32042253521126760563
Average Parents_Children_Aboard died in class 3 : 0.67660764212488350419


# Do any passengers have the same name?

In [26]:
pg_curs.execute('SELECT count(*) FROM (SELECT count(*) FROM Titanic GROUP BY Name having count(*) > 1) s')
num7 = pg_curs.fetchall()
pprint.pprint(num7)

[(887,)]


In [30]:
print("There are", num7[0][0],"name used more than once.")

There are 887 name used more than once.
